# Notebook to check direct db calls against Bovas's NAV report code

In [1]:
%reload_ext autoreload
%autoreload 
import sys, os
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root_path not in sys.path:
    sys.path.append(project_root_path)
print(project_root_path)

import datetime as dt
import numpy as np
import pandas as pd
import plotly.express as px

import pymd 
import nav.utils
from qpt_historic_pos.impl.utils.times import ChicagoTimeZone, UtcTimeZone
import qpt_stress_test.apps.generate_daily_summary as generate_daily_summary
import qpt_stress_test.services.positions as positions
import qpt_stress_test.core.config as config
import qpt_stress_test.core.qpt_config as qpt_config
import qpt_stress_test.db.tasks as db_tasks
import qpt_stress_test.db.repositories.drivers.pyodbc as pyodbc
import qpt_stress_test.db.repositories.drivers.sqlalchemy as sqlalchemy
import qpt_stress_test.db.repositories.qpt_pg as qpt_pg
import qpt_stress_test.db.repositories.qpt_mssql as qpt_mssql
import qpt_stress_test.db.repositories.databricks as db_trading
from qpt_stress_test.db.tasks import bfc_rds_sqlalchemy_engine_factory, sv_awoh_dw01_pyodbc_connection_factory,  gdt_cluster_databricks_connection_factory

from importlib import reload
reload(positions)
reload(config)
reload(qpt_config)
reload(db_tasks)

pymd.enable_logging()

c:\Users\skingham\Projects\galaxysk\qpt_stress_test


In [2]:
# Set dates, etc, for report generation
nav_date = dt.date(2023, 1, 4)
nav_00utc = dt.datetime.combine(nav_date + dt.timedelta(days=1), dt.time(hour=0, minute=0, second=0))
derivs_utc_datetime = ChicagoTimeZone.localize(dt.datetime.combine(nav_date, dt.time(hour=16, minute=0, second=0))).astimezone(UtcTimeZone)

## Get Summary Exchange Balances used in Bovas's NAV report

In [3]:
%%capture
# This code is copied from the generate_daily_summary.py app 
trading_repo = qpt_mssql.TradingRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)

# Run Bowen's Net open positions report: should match email's from Bcai and Wenbo's emails:
net_open_position_report_df = generate_daily_summary.net_open_positions(derivs_utc_datetime, trading_repo)                                                             

# Run a facsimile of the Bovas NAV report: should  match Bovas NAV Summary_Exchange_Balances_00UTC worksheet
assets_df, loans_df, summary = generate_daily_summary.generate_daily_nav_00utc(nav_date)
summary_exchange_balances_df = generate_daily_summary.summary_exchange_balances_00utc(nav_date, assets_df, loans_df, summary)

## Reformat into format to combine nav & deriv positions
asset_loans_cash_df = generate_daily_summary.summary_asset_loans_cash(summary_exchange_balances_df)
asset_and_open_positions_df = pd.concat([net_open_position_report_df, asset_loans_cash_df], ignore_index=True)

In [4]:
# Take Bovas NAV report, un-mangle accounts
# ['Account','Balance','BalanceType','Currency','Source','Timestamp','Timestamp_Native','Notional','REFERENCE 1','REFERENCE 2','TYPE','Endpoint']
reported_eod_balances = summary_exchange_balances_df.copy()
reported_eod_balances.loc[(reported_eod_balances['Balance'] < 0) & (reported_eod_balances['BalanceType'] == ''), 'BalanceType'] = 'Margin Loan'
reported_eod_balances.loc[reported_eod_balances['Balance'] < 0, 'Account'] = reported_eod_balances.loc[reported_eod_balances['Balance'] < 0, 'Account'].replace(
    ['HUBI-M Margin Loan','HUB2-M Margin Loan','WOOX-1-M-E Margin Loan','FTXE-1-M-E Margin Loan', 'OKEX-2-M Margin Loan','OKEX-2-S3 Margin Loan','BINE-2-S1-M Margin Loan'],
    ['HUBI-M','HUB2-M','WOOX-1-M-E','FTXE-1-M-E', 'OKEX-2-M','OKEX-2-S3','BINE-2-S1-M'])

print(f"Bovas Net Asset Value w/o GD Capital: {summary_exchange_balances_df.Notional.sum():,.0f}; "
      f"Total NAV & Net Open Positions balance: {asset_and_open_positions_df.notional.sum():,.0f}")

usd_currencies = ('USD', 'USDT', 'USDC')
usd_like_balances = reported_eod_balances.loc[reported_eod_balances['Currency'].isin(usd_currencies)]
balances_ex_usd = reported_eod_balances.loc[~reported_eod_balances['Currency'].isin(usd_currencies)]
print(f"  {usd_currencies} balance: {usd_like_balances.Notional.sum():,.0f};  NAV excl USD like balances: {balances_ex_usd.Notional.sum():,.0f}")

#['utc_timestamp', 'exchange', 'account', 'instrument', 'position', 'notional', 'mark_price', 'unrealized_pnl', 'instrument_type', 'expiration_time', 'is_linear', 'underlying']
nav_by_account = asset_and_open_positions_df[['exchange', 'notional']].groupby(by=['exchange']).sum().reset_index()
display(nav_by_account)

nav_by_curency = reported_eod_balances[['Timestamp', 'Currency', 'Balance', 'Notional']].groupby(by=['Timestamp', 'Currency',]).sum().reset_index()
display(nav_by_curency)

# [Timestamp], [Account], [Currency], [Source], [BalanceType]	Notional Balance
nav_by_account_ccy = reported_eod_balances[['Timestamp', 'Endpoint', 'Currency', 'Source', 'BalanceType', 'Balance', 'Notional']].groupby(by=['Timestamp', 'Endpoint', 'Currency', 'Source', 'BalanceType']).sum().reset_index()
display(nav_by_account_ccy)

Bovas Net Asset Value w/o GD Capital: 33,301,979; Total NAV & Net Open Positions balance: 13,166,647
  ('USD', 'USDT', 'USDC') balance: 21,297,699;  NAV excl USD like balances: 12,004,280


,exchange,notional
0,BINANCE,"14,559,017.85"
1,BTFX,160.12
2,BTSE,"157,674.21"
3,BULL,"48,367,717.43"
4,CASH,"699,614.89"
5,DEFI,"5,795,081.44"
6,DYDX,"-8,726.52"
7,ED&F,"76,598,881.22"
8,FBLK,"37,237,163.74"
9,GALAXY,"-189,130,431.40"


,Timestamp,Currency,Balance,Notional
0,20230104,$ PLEASE VISIT ACADY.SITE TO RECEIVE REWARD.,"180,000.00",0.00
1,20230104,$ USDCBONUS.COM <- VISIT TO CLAIM BONUS,"7,000.00",0.00
2,20230104,$ USDCGIFT.COM <- VISIT TO CLAIM BONUS,"7,000.00",0.00
3,20230104,$ USDCXMAS.COM <- VISIT TO CLAIM GIFT,"7,000.00",0.00
4,20230104,927$ VISIT AUSDBONUS.COM TO CLAIM,8.00,0.00
5,20230104,AAC,377.10,0.10
6,20230104,AAVE,192.00,"10,903.36"
7,20230104,ADA,"-16,322.80","-4,369.25"
8,20230104,ALGO,"3,628.71",678.90
9,20230104,ARPA,0.53,0.01


,Timestamp,Endpoint,Currency,Source,BalanceType,Balance,Notional
0,20230104,BINE,AAVE,bine_2_s1_e.account_asset,,569.39,"32,334.51"
1,20230104,BINE,AAVE,bine_2_s1_m.margin_account_asset,Margin Loan,-900.00,"-51,108.75"
2,20230104,BINE,ADA,bine_2_s1_e.account_asset,,"280,687.95","75,133.84"
3,20230104,BINE,ADA,bine_2_s1_m.margin_account_asset,Margin Loan,"-450,000.00","-120,454.85"
4,20230104,BINE,ALGO,bine_2_s1_e.account_asset,,"213,161.09","39,880.85"
5,20230104,BINE,ALGO,bine_2_s1_m.margin_account_asset,Margin Loan,"-244,800.00","-45,800.26"
6,20230104,BINE,ATOM,bine_2_s1_e.account_asset,,0.00,0.04
7,20230104,BINE,AVAX,bine_2_s1_e.account_asset,,"3,220.03","38,890.26"
8,20230104,BINE,AVAX,bine_2_s1_m.margin_account_asset,Margin Loan,"-6,000.00","-72,465.68"
9,20230104,BINE,BAT,bine_2_s1_e.account_asset,,"78,517.63","14,170.15"


## Reconcile Bovas totals

Let's just take the queries that Bovas uses to get crypto exchange asset & loan entries and look at all entries for 'Operations.balances.EndOfDay_00UTC'.  What's missing?

In [5]:
# Bovas config
bovas_asset_account_name_replacement = (
    ['OKEX-2-W1', 'OKEX-2-U1', 'FBLK-AUDIT-BINE', 'FBLK-AUDIT-HUBI', 'FBLK-AUDIT-OKEX', 'FBLK-Default', 'FBLK-GOTC-GACM', 'FBLK-LEND-BTGO', 'FBLK-LEND-DRAW', 
     'FBLK-LEND-OXTF', 'FBLK-LEND-XRPF', 'FBLK-LMAC-M', 'FBLK-Network Deposits', 'FBLK-PITX-E', 'FBLK-LEND-CELS', 'FBLK-LEND-GADI', 'FBLK-LEND-NICO', 
     'FBLK-LEND-GENX','FBLK-BINE-MX-S1','FBLK-DEFI-AAVE','FBLK-GOTC-BIGO','FBLK-DYDX-1-M-P','FBLK-WOOX-1-M-E','FBLK-LEND-GACM'], 
    ['OKEX-2-M-W', 'OKEX-2-M', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK','FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 
     'FBLK', 'FBLK', 'FBLK', 'FBLK'])

bovas_loan_account_name_replacement = (
    ['HUBI-M','HUB2-M','WOOX-1-M-E','FTXE-1-M-E','OKEX-2-U1','OKEX-2-S3','BINE-2-S1-M'],
    ['HUBI-M Margin Loan','HUB2-M Margin Loan','WOOX-1-M-E Margin Loan','FTXE-1-M-E Margin Loan', 
     'OKEX-2-M Margin Loan','OKEX-2-S3 Margin Loan','BINE-2-S1-M Margin Loan'])

bovas_currency_replacement = (
    ['USDT_ERC20','SRM_LOCKED','BTCUSD','ETHUSD','EOSUSD','LINKUSD','LTCUSD','ATOM.S','DOT.S','KSM.S','FTM_FANTOM','AUSDC_ETH',
     'CVXCRV-F','VARIABLEDEBTCRV','BNB_BSC','ZIL_BSC','EUROC_ETH_F5NG','USDTEST','FRXETH','AAVAUSDC','VARIABLEDEBTAVAUSDT',
     'VARIABLEDEBTUSDT','AURAB-STETH-STABLE-VAULT'],
    ['USDT','SRM','BTC','ETH','EOS','LINK','LTC','ATOM','DOT','KSM','FTM','AUSDC','CVX','CRV','BNB','ZIL','EUROC','USD','ETH',
    'AUSDC','USDT','USDT','ETH'])

In [6]:
%%capture
# Do a direct call against the database:
#
marks1_qry = """
select format(timestamp,'yyyyMMdd') as DATE, symbol as Currency, _close as Mark, 'coinmarketcap' as Source 
from RawData.cnmk_1_m.ohlcv_historical 
where format(timestamp,'yyyyMMdd') = '{date:%Y%m%d}' 
    and name not in ('CyberMiles','Genesis Mana','NFT','NEFTiPEDiA', 'UNICORN Token','DEONEX COIN','Don-key','LOLTOKEN','SoMee.Social [OLD]') 
""".format(date=nav_date)
marks1_df = pyodbc.SqlQuery(marks1_qry, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory).as_dataframe()
marks2_qry = "select DATE, Currency, Mark, Source from trading.pnl.marks where DATE = '{date:%Y%m%d}'".format(date=nav_date)
marks2_df = pyodbc.SqlQuery(marks1_qry, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory).as_dataframe()
marks_df = nav.utils._format_marks(marks1_df, marks2_df, nav_date)

bovas_loans_qry = """
select a.Account, a.Balance, a.BalanceType, UPPER(a.Currency) as Currency, 
    Isnull(b.TableName, '') as Source, FORMAT(a.Date_UTC,'yyyyMMdd') as Timestamp, a.AsOf_UTC as Timestamp_Native
from Operations.balances.EndOfDay_00UTC A 
    LEFT JOIN Operations.balances.sources B on a.Account = b.Account 
where balance != 0 
    and a.Date_UTC = '{date:%Y-%m-%d}' 
    and Currency not like '%SWAP%' 
    and a.Account in {accounts} 
    and balancetype != 'unrealized' 
order by Account
""".format(date=nav_date, accounts=tuple(qpt_config.loan_accounts))
bovas_loans_df = pyodbc.SqlQuery(bovas_loans_qry, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory).as_dataframe()

bovas_assets_qry = """
select a.Account, a.Balance, a.BalanceType, UPPER(a.Currency) as Currency,  
    Isnull(b.TableName, '') as Source, FORMAT(a.Date_UTC,'yyyyMMdd') as Timestamp, a.AsOf_UTC as Timestamp_Native 
from Operations.balances.EndOfDay_00UTC A 
    LEFT JOIN Operations.balances.sources B on a.Account = b.Account 
where balance != 0 and a.Date_UTC = '{date:%Y-%m-%d}' 
    and Currency not like '%SWAP%' 
    and a.Account in {accounts} 
    and Seconds_from_00UTC <= 79259
order by Account
""".format(date=nav_date, accounts=tuple(qpt_config.exchange_balance_accounts))
bovas_assets_df = pyodbc.SqlQuery(bovas_assets_qry, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory).as_dataframe()

op_eod_balances_df = trading_repo.get_operations_eod_balances(nav_date).as_dataframe()
op_eod_balances_df.Currency.replace(*bovas_currency_replacement, inplace=True)
op_eod_balances_df['Notional'] = op_eod_balances_df.Balance * marks_df.reindex(op_eod_balances_df.Currency).fillna(0).Mark.values


In [13]:
# display(marks_df)
# display(bovas_loans_df)
# display(bovas_assets_df)
# display(op_eod_balances_df)

# Display DF of all records not used in Bovas's report
op_eod = op_eod_balances_df.copy()
bovas_assets_eod = bovas_assets_df.copy()
bovas_assets_eod = nav.utils._format_assets(nav_date, bovas_assets_eod, marks_df)
bovas_loans_eod = bovas_loans_df.copy()
bovas_loans_eod = nav.utils._format_loans(nav_date, bovas_loans_eod, marks_df)

# Assets - apply the mappings to account names that Bovas's code does
op_eod.loc[((op_eod['Balance'] > 0) | (op_eod['BalanceType'] == 'Unrealized')), 'Account'] \
      = op_eod.loc[((op_eod['Balance'] > 0) | (op_eod['BalanceType'] == 'Unrealized')), 'Account'].replace(*bovas_asset_account_name_replacement)
#bovas_assets_eod.Account.replace(*bovas_asset_account_name_replacement, inplace=True)
#bovas_assets_eod.Currency.replace(*bovas_currency_replacement, inplace=True)

# Loans
op_eod.loc[((op_eod['Balance'] < 0) & (op_eod['BalanceType'] != 'Unrealized')), 'Account'] \
      = op_eod.loc[((op_eod['Balance'] < 0) & (op_eod['BalanceType'] != 'Unrealized')), 'Account'].replace(*bovas_loan_account_name_replacement)
#bovas_loans_eod.Account.replace(*bovas_loan_account_name_replacement, inplace=True)
#bovas_loans_eod.Currency.replace(*bovas_currency_replacement, inplace=True)

# Sum Fireblocks to currency
op_eod = nav.utils._format_fireblocks(op_eod)

df1 = pd.concat([op_eod, bovas_assets_eod, bovas_loans_eod]).set_index(op_eod.columns.tolist())
df2 = pd.concat([bovas_assets_eod, bovas_loans_eod]).set_index(op_eod.columns.tolist())
bovas_ignored_df = df1.loc[df2.index.symmetric_difference(df1.index)].reset_index()

# Get all the bovas cypto position records, without the account name mangling, into one df
#op_eod = op_eod_balances_df.copy()
df1 = pd.concat([op_eod, bovas_ignored_df]).set_index(op_eod.columns.tolist())
df2 = bovas_ignored_df.set_index(op_eod.columns.tolist())
bovas_included_df = df1.loc[df2.index.symmetric_difference(df1.index)].reset_index()

print(f"Total Exchange EOD notional balance reported in Bovas's report:  {bovas_included_df.Notional.sum():,.0f}"
      f"\tNotional notional balance of EOD records removed from reporting: {bovas_ignored_df.Notional.sum():,.0f}\n")

# Three views of the exchange positions
print("Summary from Bovas's code")
reported_eod_df = reported_eod_balances.copy()
reported_eod_df['Unrealized'] = reported_eod_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
reported_eod_df['Loans-funding'] = reported_eod_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
reported_eod_df['Assets'] = reported_eod_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
report_by_exchange = reported_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Endpoint',]).sum().reset_index()
display(report_by_exchange)

print("Summary from ops table")
op_eod_df = op_eod.copy()
op_eod_df['Exchange'] = op_eod_df['Account'].apply(lambda ac: str(ac.split('-')[0]))
op_eod_df['Unrealized'] = op_eod_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
op_eod_df['Loans-funding'] = op_eod_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
op_eod_df['Assets'] = op_eod_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
op_eod_by_exchange = op_eod_df[['Timestamp', 'Exchange', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Exchange',]).sum().reset_index()
display(op_eod_by_exchange)

print("Summary from bovas's sql")
bovas_df = pd.concat([bovas_assets_eod, bovas_loans_eod], ignore_index=True)
bovas_df['Exchange'] = bovas_df['Account'].apply(lambda ac: str(ac.split('-')[0]))
bovas_df['Unrealized'] = bovas_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
bovas_df['Loans-funding'] = bovas_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
bovas_df['Assets'] = bovas_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
bovas_by_exchange = bovas_df[['Timestamp', 'Exchange', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Exchange',]).sum().reset_index()
display(bovas_by_exchange)

Total Exchange EOD notional balance reported in Bovas's report:  -8,721,431	Notional notional balance of EOD records removed from reporting: 33,465,665

False


c:\Users\skingham\Projects\galaxysk\qpt_stress_test\nav\utils.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp, ignore_index=True)
c:\Users\skingham\Projects\galaxysk\qpt_stress_test\nav\utils.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp, ignore_index=True)


,Timestamp,Endpoint,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","69,256,054.93","-265,009.95","67,373,181.20"
1,20230104,BTFX,0.00,160.12,0.00,160.12
2,20230104,BTSE,0.00,"157,674.21",0.00,"157,674.21"
3,20230104,BULL,"-68,606,421.63","116,974,139.05",0.00,"48,367,717.43"
4,20230104,CASH,0.00,"699,614.89",0.00,"699,614.89"
5,20230104,DEFI,"-10,018,285.23","15,813,366.67",0.00,"5,795,081.44"
6,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
7,20230104,ED&F,0.00,"31,046,831.22",0.00,"31,046,831.22"
8,20230104,FBLK,0.00,"37,237,163.74",0.00,"37,237,163.74"
9,20230104,GALAXY,"-203,819,937.55","14,689,506.15",0.00,"-189,130,431.40"


False


,Timestamp,Exchange,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","68,409,427.32","-265,009.95","66,526,553.58"
1,20230104,BTFX,0.00,160.10,0.00,160.10
2,20230104,BTSE,0.00,"42,937.37",0.00,"42,937.37"
3,20230104,BULL,0.00,"65,074,533.48",0.00,"65,074,533.48"
4,20230104,DEFI,"-10,018,285.23","39,106,228.23",0.00,"29,087,943.00"
5,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
6,20230104,FBLK,0.00,"59,532,499.87",0.00,"59,532,499.87"
7,20230104,FUB2,0.00,20.34,0.00,20.34
8,20230104,FUBI,0.00,638.67,0.00,638.67
9,20230104,GATE,0.00,"20,648.72",0.00,"20,648.72"


,Timestamp,Exchange,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","68,409,427.32","-265,009.95","66,526,553.58"
1,20230104,BTFX,0.00,160.10,0.00,160.10
2,20230104,BTSE,0.00,"42,937.37",0.00,"42,937.37"
3,20230104,BULL,0.00,"65,074,533.48",0.00,"65,074,533.48"
4,20230104,DEFI,"-10,018,285.23","15,813,366.67",0.00,"5,795,081.44"
5,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
6,20230104,FBLK,0.00,"37,237,163.74",0.00,"37,237,163.74"
7,20230104,FUB2,0.00,20.34,0.00,20.34
8,20230104,FUBI,0.00,638.67,0.00,638.67
9,20230104,GATE,0.00,"20,648.72",0.00,"20,648.72"


In [12]:

print("Operations.balances.EndOfDay_00UTC records that are not included in Bovas's NAV report:")
#display(bovas_ignored_df.sort_values(by=['Account', 'Currency'], ignore_index=True))

print("\nOperations.balances.EndOfDay_00UTC records that are included in Bovas's NAV report:")
#display(bovas_included_df.sort_values(by=['Account', 'Currency'], ignore_index=True))

False


,Timestamp,Endpoint,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","69,256,054.93","-265,009.95","67,373,181.20"
1,20230104,BTFX,0.00,160.12,0.00,160.12
2,20230104,BTSE,0.00,"157,674.21",0.00,"157,674.21"
3,20230104,BULL,"-68,606,421.63","116,974,139.05",0.00,"48,367,717.43"
4,20230104,CASH,0.00,"699,614.89",0.00,"699,614.89"
5,20230104,DEFI,"-10,018,285.23","15,813,366.67",0.00,"5,795,081.44"
6,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
7,20230104,ED&F,0.00,"31,046,831.22",0.00,"31,046,831.22"
8,20230104,FBLK,0.00,"37,237,163.74",0.00,"37,237,163.74"
9,20230104,GALAXY,"-203,819,937.55","14,689,506.15",0.00,"-189,130,431.40"


False


,Timestamp,Exchange,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","68,409,427.32","-265,009.95","66,526,553.58"
1,20230104,BTFX,0.00,160.10,0.00,160.10
2,20230104,BTSE,0.00,"42,937.37",0.00,"42,937.37"
3,20230104,BULL,0.00,"65,074,533.48",0.00,"65,074,533.48"
4,20230104,DEFI,"-10,018,285.23","39,106,228.23",0.00,"29,087,943.00"
5,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
6,20230104,FBLK,0.00,"59,532,499.87",0.00,"59,532,499.87"
7,20230104,FUB2,0.00,20.34,0.00,20.34
8,20230104,FUBI,0.00,638.67,0.00,638.67
9,20230104,GATE,0.00,"20,648.72",0.00,"20,648.72"


,Timestamp,Exchange,Loans-funding,Assets,Unrealized,Notional
0,20230104,BINE,"-1,617,863.78","68,409,427.32","-265,009.95","66,526,553.58"
1,20230104,BTFX,0.00,160.10,0.00,160.10
2,20230104,BTSE,0.00,"42,937.37",0.00,"42,937.37"
3,20230104,BULL,0.00,"65,074,533.48",0.00,"65,074,533.48"
4,20230104,DEFI,"-10,018,285.23","15,813,366.67",0.00,"5,795,081.44"
5,20230104,DYDX,0.00,"19,411.58",0.00,"19,411.58"
6,20230104,FBLK,0.00,"37,237,163.74",0.00,"37,237,163.74"
7,20230104,FUB2,0.00,20.34,0.00,20.34
8,20230104,FUBI,0.00,638.67,0.00,638.67
9,20230104,GATE,0.00,"20,648.72",0.00,"20,648.72"


## Compare the Ops EOD balance table with the balances reported in the account balance records